In [1]:
%load_ext autoreload
%matplotlib inline
%autoreload 2

In [2]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import pickle
import h5py

In [3]:
sys.path.append("..")

from WatChMaL.analysis.multi_plot_utils import multi_disp_learn_hist, multi_compute_roc, multi_plot_roc
from WatChMaL.analysis.comparison_utils import multi_get_masked_data, multi_collapse_test_output
from WatChMaL.analysis.fitqun_comparison_utils import load_fq_output
from WatChMaL.analysis.performance_analysis_utils import plot_fitqun_binned_performance, plot_fixed_operating_performance

In [4]:
############# define plotting params #############
c = plt.rcParams['axes.prop_cycle'].by_key()['color']
label_dict = {"$\gamma$":0, "$e$":1, "$\mu$":2}
inverse_label_dict = {0:"$\gamma$", 1:"$e$", 2:"$\mu$"}
muon_softmax_index_dict = {"e/gamma":0, "mu":1}

## Load fitqun results

In [5]:
# Retrieve flags
gamma_file_path = '/fast_scratch/WatChMaL/data/IWCD_mPMT_Short_gamma_E0to1000MeV_unif-pos-R400-y300cm_4pi-dir.fiTQun.root'
e_file_path     = '/fast_scratch/WatChMaL/data/IWCD_mPMT_Short_e-_E0to1000MeV_unif-pos-R400-y300cm_4pi-dir.fiTQun.root'
mu_file_path    = '/fast_scratch/WatChMaL/data/IWCD_mPMT_Short_mu-_E0to1000MeV_unif-pos-R400-y300cm_4pi-dir.fiTQun.root'

In [6]:
fq_mapping_path = '/home/jtindall/WatChMaL_analysis/fitqun_comparison/fq_comparison_data/3M_fitqun_mapping.pickle'

In [7]:
fq_scores, fq_labels, fq_mom = load_fq_output(fq_mapping_path, gamma_file_path, e_file_path, mu_file_path)

In [8]:
print(fq_mom)

[206.95685 842.6756  233.17891 ... 361.7926  144.14459 717.5251 ]


## Load resnet results

In [9]:
############# define run locations #############
short_locs = ['/home/jtindall/WatChMaL/outputs/2021-03-16/20-21-15/outputs']

short_titles = ['Short Tank', 'Fitqun']

full_locs = []

full_titles = []

short_linecolor = [c[0] for _ in range(2)]
short_linestyle = ['--' for _ in range(2)]

full_linecolor = [c[1] for _ in full_locs]
full_linestyle = [':' for _ in full_locs]


In [10]:
############# load short tank run data #############
short_raw_output_softmax    = [np.load(loc + "/softmax.npy") for loc in short_locs]
short_raw_actual_labels     = [np.load(loc + "/labels.npy") for loc in short_locs]

In [11]:
locs      = short_locs + full_locs
titles    = short_titles + full_titles
linecolor = short_linecolor + full_linecolor
linestyle = short_linestyle + full_linestyle

## Add fitqun results

In [12]:
short_raw_output_softmax = short_raw_output_softmax + [fq_scores]
short_raw_actual_labels  = short_raw_actual_labels + [fq_labels]

## Filter results

In [13]:
def remove_indices(array, cut_idxs):
    return np.delete(array, cut_idxs, 0)

with open('./fq_comparison_data/3M_fitqun_cuts.pickle', 'rb') as handle:
    cuts = pickle.load(handle)

cut_to_apply = cuts['fq_comparison']

In [14]:
short_softmax = [remove_indices(softmax, cut_to_apply) for softmax in short_raw_output_softmax]
short_labels  = [remove_indices(labels, cut_to_apply) for labels in short_raw_actual_labels]

## Load h5 Data

In [15]:
# Import test events from h5 file
data_path = "/fast_scratch/WatChMaL/data/IWCD_mPMT_Short_emg_E0to1000MeV_digihits.h5"
data_file = h5py.File(data_path, "r")

print(data_file.keys())

energies   = np.array(data_file['energies'])
angles     = np.array(data_file['angles'])
positions  = np.array(data_file['positions'])
labels     = np.array(data_file['labels'])

<KeysViewHDF5 ['angles', 'energies', 'event_hits_index', 'event_ids', 'hit_charge', 'hit_pmt', 'hit_time', 'labels', 'positions', 'root_files', 'veto', 'veto2']>


In [16]:
idxs_path = '/fast_scratch/WatChMaL/data/IWCD_mPMT_Short_3M_idxs.npz'
idxs = np.load(idxs_path, allow_pickle=True)

test_idxs  = idxs['test_idxs']

test_angles    = angles[test_idxs]
test_energies  = energies[test_idxs]
test_positions = positions[test_idxs]
test_labels    = labels[test_idxs]
test_info = {'name':'Test Set','energies':test_energies,'positions':test_positions, 'angles':test_angles, 'labels':test_labels}

print("Done test info")

Done test info


In [17]:
filtered_test_energies  = remove_indices(test_energies, cut_to_apply)
filtered_test_angles    = remove_indices(test_angles, cut_to_apply)
filtered_test_zenith    = filtered_test_angles[:, 0]
filtered_test_azimuth   = filtered_test_angles[:, 1]

## Load Parameters

In [18]:
filtered_fq_mom = remove_indices(fq_mom, cut_to_apply)

In [19]:
d_to_wall = np.load('./fq_comparison_data/3M_d_to_wall.npz', allow_pickle=True)

test_d_to_wall  = d_to_wall['test_d_to_wall']

filtered_test_d_to_wall = remove_indices(test_d_to_wall, cut_to_apply)

In [20]:
true_mom = np.load('./fq_comparison_data/3M_momenta.npz', allow_pickle=True)

test_true_mom  = true_mom['test_momenta']

filtered_test_true_mom = remove_indices(test_true_mom, cut_to_apply)

In [21]:
print(test_energies.shape)

(3524482, 1)


In [22]:
# bin energy data
energy_hist, energy_bins = np.histogram(filtered_test_energies, bins=9)
print("Energy bins: ", energy_bins)

# bin d to wall data
d_to_wall_hist, d_to_wall_bins = np.histogram(filtered_test_d_to_wall, bins=9)
print("Distance to wall bins: ", d_to_wall_bins)


Energy bins:  [   1.4282632  130.18549    258.94272    387.69995    516.45715
  645.2144     773.9716     902.7288    1031.4861    1160.2433   ]
Distance to wall bins:  [ 50.09834499 147.02997846 243.96161194 340.89324541 437.82487889
 534.75651236 631.68814584 728.61977932 825.55141279 922.48304627]


## Binned Performance Plots

### Plot vs Energy

In [23]:
# Plot vs Energy
plot_binned_performance(softmaxes = short_softmax[0],
                        labels    = short_labels[0],
                        binning_features = filtered_test_energies,
                        binning_label = 'Energy',
                        operating_point    = 0.005,
                        bins = energy_bins[:-1],
                        index_dict = label_dict,
                        label_0 = "$e$", 
                        label_1 = "$\gamma$",
                        metric='efficiency',
                        fixed='rejection')

NameError: name 'plot_binned_performance' is not defined

In [ ]:
fig, axes = plt.subplots(1,1,figsize=(9,5), facecolor='w')

plot_fixed_operating_performance(scores                 = short_softmax[0][:,0] + short_softmax[0][:,1], 
                                 labels                 = short_labels[0],  
                                 fixed_binning_features = filtered_fq_mom,
                                 fixed_bin_label        = 'Reconstructed Momentum',
                                 plot_binning_features  = filtered_test_energies,
                                 plot_bin_label         = 'True Energy',
                                 fpr_fixed_point        = 0.001, 
                                 index_dict             = label_dict, 
                                 fixed_bin_size         = 50, 
                                 plot_bins              = 7, 
                                 ax                     = axes,
                                 marker                 = 'o--',
                                 color                  = 'r',
                                 title_note             = '',
                                 metric                 = 'efficiency',
                                yrange                  = [0.85, 1.01]
                              )

plot_fixed_operating_performance(scores                 = short_softmax[1][:,0] + short_softmax[1][:,1], 
                                 labels                 = short_labels[1],   
                                 fixed_binning_features = filtered_fq_mom,
                                 fixed_bin_label        = 'Reconstructed Momentum',
                                 plot_binning_features  = filtered_test_energies,
                                 plot_bin_label         = 'True Energy',
                                 fpr_fixed_point        = 0.001, 
                                 index_dict             = label_dict, 
                                 fixed_bin_size         = 50, 
                                 plot_bins              = 7, 
                                 ax                     = axes,
                                 marker                 = 'o--',
                                 color                  = 'g',
                                 title_note             = '',
                                 metric                 = 'efficiency',
                                yrange                  = [0.85, 1.01]
                              )
axes.legend(['ResNet (P(e-) + P(\u03B3))', 'FiTQun'])

In [ ]:
fig, axes = plt.subplots(1,1,figsize=(9,5), facecolor='w')

plot_fixed_operating_performance(scores                 = short_softmax[0][:,0] + short_softmax[0][:,1], 
                                 labels                 = short_labels[0],  
                                 fixed_binning_features = filtered_fq_mom,
                                 fixed_bin_label        = 'Reconstructed Momentum',
                                 plot_binning_features  = filtered_test_energies,
                                 plot_bin_label         = 'True Energy',
                                 fpr_fixed_point        = 0.005, 
                                 index_dict             = label_dict, 
                                 fixed_bin_size         = 50, 
                                 plot_bins              = 7, 
                                 ax                     = axes,
                                 marker                 = 'o-',
                                 color                  = 'r',
                                 title_note             = '',
                                 metric                 = 'efficiency',
                                yrange                  = [0.85, 1.01]
                              )

plot_fixed_operating_performance(scores                 = short_softmax[0][:,0] + short_softmax[0][:,1], 
                                 labels                 = short_labels[0],  
                                 fixed_binning_features = filtered_fq_mom,
                                 fixed_bin_label        = 'Reconstructed Momentum',
                                 plot_binning_features  = filtered_test_energies,
                                 plot_bin_label         = 'True Energy',
                                 fpr_fixed_point        = 0.001, 
                                 index_dict             = label_dict, 
                                 fixed_bin_size         = 50, 
                                 plot_bins              = 7, 
                                 ax                     = axes,
                                 marker                 = 'o--',
                                 color                  = 'r',
                                 title_note             = '',
                                 metric                 = 'efficiency',
                                yrange                  = [0.85, 1.01]
                              )

plot_fixed_operating_performance(scores                 = short_softmax[1][:,0] + short_softmax[1][:,1], 
                                 labels                 = short_labels[1],   
                                 fixed_binning_features = filtered_fq_mom,
                                 fixed_bin_label        = 'Reconstructed Momentum',
                                 plot_binning_features  = filtered_test_energies,
                                 plot_bin_label         = 'True Energy',
                                 fpr_fixed_point        = 0.005, 
                                 index_dict             = label_dict, 
                                 fixed_bin_size         = 50, 
                                 plot_bins              = 7, 
                                 ax                     = axes,
                                 marker                 = 'o-',
                                 color                  = 'g',
                                 title_note             = '',
                                 metric                 = 'efficiency',
                                yrange                  = [0.85, 1.01]
                              )

plot_fixed_operating_performance(scores                 = short_softmax[1][:,0] + short_softmax[1][:,1], 
                                 labels                 = short_labels[1],   
                                 fixed_binning_features = filtered_fq_mom,
                                 fixed_bin_label        = 'Reconstructed Momentum',
                                 plot_binning_features  = filtered_test_energies,
                                 plot_bin_label         = 'True Energy',
                                 fpr_fixed_point        = 0.001, 
                                 index_dict             = label_dict, 
                                 fixed_bin_size         = 50, 
                                 plot_bins              = 7, 
                                 ax                     = axes,
                                 marker                 = 'o--',
                                 color                  = 'g',
                                 title_note             = '',
                                 metric                 = 'efficiency',
                                yrange                  = [0.85, 1.01]
                              )

axes.legend(['ResNet, Rejection 200', 'ResNet, Rejection 1000', 'FiTQun,  Rejection 200', 'FiTQun,  Rejection 1000', 'FiTQun, no binning', 'FiTQun', 'FiTQun, no binning'])

### Plot vs To Wall

In [ ]:
fig, axes = plt.subplots(1,1,figsize=(9,5), facecolor='w')

plot_fixed_operating_performance(scores                 = short_softmax[0][:,0] + short_softmax[0][:,1], 
                                 labels                 = short_labels[0],  
                                 fixed_binning_features = filtered_fq_mom,
                                 fixed_bin_label        = 'Reconstructed Momentum',
                                 plot_binning_features  = filtered_test_d_to_wall,
                                 plot_bin_label         = 'To Wall',
                                 fpr_fixed_point        = 0.005, 
                                 index_dict             = label_dict, 
                                 fixed_bin_size         = 50, 
                                 plot_bins              = 20, 
                                 ax                     = axes,
                                 marker                 = 'o-',
                                 color                  = 'r',
                                 title_note             = '',
                                 metric                 = 'efficiency',
                                yrange                  = [0.7, 1.01]
                              )

plot_fixed_operating_performance(scores                 = short_softmax[0][:,0] + short_softmax[0][:,1], 
                                 labels                 = short_labels[0],  
                                 fixed_binning_features = filtered_fq_mom,
                                 fixed_bin_label        = 'Reconstructed Momentum',
                                 plot_binning_features  = filtered_test_d_to_wall,
                                 plot_bin_label         = 'To Wall',
                                 fpr_fixed_point        = 0.001, 
                                 index_dict             = label_dict, 
                                 fixed_bin_size         = 50, 
                                 plot_bins              = 20, 
                                 ax                     = axes,
                                 marker                 = 'o--',
                                 color                  = 'r',
                                 title_note             = '',
                                 metric                 = 'efficiency',
                                yrange                  = [0.7, 1.01]
                              )

"""
plot_fixed_operating_performance(scores                 = short_softmax[0][:,0] + short_softmax[0][:,1], 
                                 labels                 = short_labels[0],  
                                 fixed_binning_features = filtered_fq_mom,
                                 fixed_bin_label        = 'Reconstructed Momentum',
                                 plot_binning_features  = filtered_test_d_to_wall,
                                 plot_bin_label         = 'To Wall',
                                 fpr_fixed_point        = 0.005, 
                                 index_dict             = label_dict, 
                                 fixed_bin_size         = 610, 
                                 plot_bins              = 20, 
                                 ax                     = axes,
                                 marker                 = 'o--',
                                 color                  = 'k',
                                 title_note             = '',
                                 metric                 = 'efficiency',
                                yrange                  = [0.7, 1.01]
                              )
"""

plot_fixed_operating_performance(scores                 = short_softmax[1][:,0] + short_softmax[1][:,1], 
                                 labels                 = short_labels[1],  
                                 fixed_binning_features = filtered_fq_mom,
                                 fixed_bin_label        = 'Reconstructed Momentum',
                                 plot_binning_features  = filtered_test_d_to_wall,
                                 plot_bin_label         = 'To Wall',
                                 fpr_fixed_point        = 0.005, 
                                 index_dict             = label_dict, 
                                 fixed_bin_size         = 50, 
                                 plot_bins              = 20, 
                                 ax                     = axes,
                                 marker                 = 'o-',
                                 color                  = 'g',
                                 title_note             = '',
                                 metric                 = 'efficiency',
                                yrange                  = [0.7, 1.01]
                              )

plot_fixed_operating_performance(scores                 = short_softmax[1][:,0] + short_softmax[1][:,1], 
                                 labels                 = short_labels[1],  
                                 fixed_binning_features = filtered_fq_mom,
                                 fixed_bin_label        = 'Reconstructed Momentum',
                                 plot_binning_features  = filtered_test_d_to_wall,
                                 plot_bin_label         = 'To Wall',
                                 fpr_fixed_point        = 0.001, 
                                 index_dict             = label_dict, 
                                 fixed_bin_size         = 50, 
                                 plot_bins              = 20, 
                                 ax                     = axes,
                                 marker                 = 'o--',
                                 color                  = 'g',
                                 title_note             = '',
                                 metric                 = 'efficiency',
                                yrange                  = [0.7, 1.01]
                              )
"""
plot_fixed_operating_performance(scores                 = short_softmax[1][:,0] + short_softmax[1][:,1], 
                                 labels                 = short_labels[1],  
                                 fixed_binning_features = filtered_fq_mom,
                                 fixed_bin_label        = 'Reconstructed Momentum',
                                 plot_binning_features  = filtered_test_d_to_wall,
                                 plot_bin_label         = 'To Wall',
                                 fpr_fixed_point        = 0.005, 
                                 index_dict             = label_dict, 
                                 fixed_bin_size         = 2000, 
                                 plot_bins              = 20, 
                                 ax                     = axes,
                                 marker                 = 'o--',
                                 color                  = 'b',
                                 title_note             = '',
                                 metric                 = 'efficiency',
                                yrange                  = [0.7, 1.01]
                              )
"""
axes.legend(['ResNet, Rejection 200', 'ResNet, Rejection 1000', 'FiTQun,  Rejection 200', 'FiTQun,  Rejection 1000', 'FiTQun, no binning', 'FiTQun', 'FiTQun, no binning'])

In [ ]:
fig, axes = plt.subplots(1,1,figsize=(9,5), facecolor='w')

plot_fixed_operating_performance(scores                 = short_softmax[0][:,0] + short_softmax[0][:,1], 
                                 labels                 = short_labels[0],  
                                 fixed_binning_features = filtered_fq_mom,
                                 fixed_bin_label        = 'Reconstructed Momentum',
                                 plot_binning_features  = filtered_fq_mom,
                                 plot_bin_label         = 'To Wall',
                                 fpr_fixed_point        = 0.005, 
                                 index_dict             = label_dict, 
                                 fixed_bin_size         = 50, 
                                 plot_bins              = 20, 
                                 ax                     = axes,
                                 marker                 = 'o--',
                                 color                  = 'r',
                                 title_note             = '',
                                 metric                 = 'efficiency',
                                 yrange                  = [0.7, 1.01]
                              )

plot_fixed_operating_performance(scores                 = short_softmax[0][:,0] + short_softmax[0][:,1], 
                                 labels                 = short_labels[0],  
                                 fixed_binning_features = filtered_fq_mom,
                                 fixed_bin_label        = 'Reconstructed Momentum',
                                 plot_binning_features  = filtered_fq_mom,
                                 plot_bin_label         = 'To Wall',
                                 fpr_fixed_point        = 0.005, 
                                 index_dict             = label_dict, 
                                 fixed_bin_size         = 610, 
                                 plot_bins              = 20, 
                                 ax                     = axes,
                                 marker                 = 'o--',
                                 color                  = 'k',
                                 title_note             = '',
                                 metric                 = 'efficiency',
                                 yrange                  = [0.7, 1.01]
                              )

plot_fixed_operating_performance(scores                 = short_softmax[1][:,0] + short_softmax[1][:,1], 
                                 labels                 = short_labels[1],  
                                 fixed_binning_features = filtered_fq_mom,
                                 fixed_bin_label        = 'Reconstructed Momentum',
                                 plot_binning_features  = filtered_fq_mom,
                                 plot_bin_label         = 'To Wall',
                                 fpr_fixed_point        = 0.005, 
                                 index_dict             = label_dict, 
                                 fixed_bin_size         = 50, 
                                 plot_bins              = 20, 
                                 ax                     = axes,
                                 marker                 = 'o--',
                                 color                  = 'g',
                                 title_note             = '',
                                 metric                 = 'efficiency',
                                yrange                  = [0.7, 1.01]
                              )

plot_fixed_operating_performance(scores                 = short_softmax[1][:,0] + short_softmax[1][:,1], 
                                 labels                 = short_labels[1],  
                                 fixed_binning_features = filtered_fq_mom,
                                 fixed_bin_label        = 'Reconstructed Momentum',
                                 plot_binning_features  = filtered_fq_mom,
                                 plot_bin_label         = 'To Wall',
                                 fpr_fixed_point        = 0.005, 
                                 index_dict             = label_dict, 
                                 fixed_bin_size         = 2000, 
                                 plot_bins              = 20, 
                                 ax                     = axes,
                                 marker                 = 'o--',
                                 color                  = 'b',
                                 title_note             = '',
                                 metric                 = 'efficiency',
                                yrange                  = [0.7, 1.01]
                              )

axes.legend(['ResNet (P(e-) + P(\u03B3))', 'ResNet (P(e-) + P(\u03B3)), no binning', 'FiTQun', 'FiTQun, no binning'])

In [ ]:
test_N = 10

fig, axes = plt.subplots(1,2,figsize=(18,9), facecolor='w')

ax = axes[0]

plot_fitqun_binned_performance(scores                 = short_softmax[1][:,1], 
                               labels                 = short_labels[1], 
                               reconstructed_momentum = filtered_fq_mom,
                               fpr_fixed_point        = 0.005, 
                               index_dict             = label_dict, 
                               recons_mom_bin_size    = 50,
                               plot_bins              = test_N,
                               ax                     = ax,
                               marker                 = 'o--',
                               color                  = 'g',
                               title_note             = '',
                               metric                 = 'efficiency',
                               yrange                 = [0.7, 1.1],
                               #xrange                 = [0, 1000]
                              )


plot_fitqun_binned_performance(scores                 = short_softmax[1][:,1], 
                               labels                 = short_labels[1], 
                               reconstructed_momentum = filtered_fq_mom,
                               fpr_fixed_point        = 0.005, 
                               index_dict             = label_dict, 
                               recons_mom_bin_size    = 610,
                               plot_bins              = test_N,
                               ax                     = ax,
                               marker                 = 'o--',
                               color                  = 'k',
                               title_note             = '',
                               metric                 = 'efficiency',
                               yrange                 = [0.7, 1.1],
                               #xrange                 = [0, 1000]
                              )

plot_fitqun_binned_performance(scores                 = short_softmax[0][:,0] + short_softmax[0][:,1], 
                               labels                 = short_labels[0],
                               reconstructed_momentum = filtered_fq_mom,
                               fpr_fixed_point        = 0.005, 
                               index_dict             = label_dict, 
                               recons_mom_bin_size    = 50, 
                               plot_bins              = test_N,
                               ax                     = ax,
                               marker                 = 'o--',
                               color                  = 'r',
                               title_note             = '',
                               metric                 = 'efficiency',
                               yrange                 = [0.7, 1.1],
                               #xrange                 = [0, 1000]
                              )

plot_fitqun_binned_performance(scores                 = short_softmax[0][:,0] + short_softmax[0][:,1], 
                               labels                 = short_labels[0],
                               reconstructed_momentum = filtered_fq_mom,
                               fpr_fixed_point        = 0.005, 
                               index_dict             = label_dict, 
                               recons_mom_bin_size    = 610, 
                               plot_bins              = test_N,
                               ax                     = ax,
                               marker                 = 'o--',
                               color                  = 'b',
                               title_note             = '',
                               metric                 = 'efficiency',
                               yrange                 = [0.7, 1.1],
                               #xrange                 = [0, 1000]
                              )


ax.legend(['ResNet (P(e-) + P(\u03B3))', 'ResNet (P(e-) + P(\u03B3)), no binning', 'FiTQun', 'FiTQun, no binning'])

ax = axes[1]

plot_fitqun_binned_performance(scores                 = short_softmax[0][:,0] + short_softmax[0][:,1], 
                               labels                 = short_labels[0],
                               reconstructed_momentum = filtered_fq_mom,
                               fpr_fixed_point        = 0.005, 
                               index_dict             = label_dict, 
                               recons_mom_bin_size    = 50,
                               plot_bins              = test_N,
                               ax                     = ax,
                               marker                 = 'o--',
                               color                  = 'r',
                               title_note             = '',
                               metric                 = 'mu fpr',
                               yrange                 = [0, 0.01],
                               #xrange                 = [0, 1000]
                                         )


plot_fitqun_binned_performance(scores                 = short_softmax[0][:,0] + short_softmax[0][:,1], 
                               labels                 = short_labels[0],
                               reconstructed_momentum = filtered_fq_mom,
                               fpr_fixed_point        = 0.005, 
                               index_dict             = label_dict, 
                               recons_mom_bin_size    = 610,
                               plot_bins              = test_N,
                               ax                     = ax,
                               marker                 = 'o--',
                               color                  = 'k',
                               title_note             = '',
                               metric                 = 'mu fpr',
                               yrange                 = [0, 0.01],
                               #xrange                 = [0, 1000]
                                         )

plot_fitqun_binned_performance(scores                 = short_softmax[1][:,1], 
                               labels                 = short_labels[1],
                               reconstructed_momentum = filtered_fq_mom,
                               fpr_fixed_point        = 0.005, 
                               index_dict             = label_dict, 
                               recons_mom_bin_size    = 50, 
                               plot_bins              = test_N,
                               ax                     = ax,
                               marker                 = 'o--',
                               color                  = 'g',
                               title_note             = '',
                               metric                 = 'mu fpr',
                               yrange                 = [0, 0.01],
                               #xrange                 = [0, 1000]
                                         )

plot_fitqun_binned_performance(scores                 = short_softmax[1][:,1], 
                               labels                 = short_labels[1],
                               reconstructed_momentum = filtered_fq_mom,
                               fpr_fixed_point        = 0.005, 
                               index_dict             = label_dict, 
                               recons_mom_bin_size    = 610, 
                               plot_bins              = test_N,
                               ax                     = ax,
                               marker                 = 'o--',
                               color                  = 'b',
                               title_note             = '',
                               metric                 = 'mu fpr',
                               yrange                 = [0, 0.01],
                               #xrange                 = [0, 1000]
                                         )

ax.legend(['ResNet (P(e-) + P(\u03B3))', 'ResNet (P(e-) + P(\u03B3)), no binning', 'FiTQun', 'FiTQun, no binning'])

In [ ]:
sample_softmax = np.zeros_like(short_softmax[0])
sample_softmax[:,1] = short_softmax[0][:,0] + short_softmax[0][:,1]
sample_softmax = sample_softmax[:, 1]

In [ ]:
print(sample_softmax.shape)

In [ ]:
print()

In [ ]:
fig, axes = plt.subplots(1,1,figsize=(9,5), facecolor='w')

plot_fixed_operating_performance(scores                 = sample_softmax, 
                                 labels                 = short_labels[0],  
                                 fixed_binning_features = filtered_fq_mom,
                                 fixed_bin_label        = 'Reconstructed Momentum',
                                 plot_binning_features  = filtered_test_d_to_wall,
                                 plot_bin_label         = 'To Wall',
                                 fpr_fixed_point        = 0.005, 
                                 index_dict             = label_dict, 
                                 fixed_bin_size         = 610, 
                                 plot_bins              = 20, 
                                 ax                     = axes,
                                 marker                 = 'o--',
                                 color                  = 'k',
                                 title_note             = '',
                                 metric                 = 'efficiency',
                                #yrange                  = [0.7, 1.01]
                              )

axes.legend(['ResNet (P(e-) + P(\u03B3))', 'ResNet (P(e-) + P(\u03B3)), no binning', 'FiTQun', 'FiTQun, no binning'])

In [ ]:
print(short_softmax[0][:,0] + short_softmax[0][:,1])

In [ ]:
print(len(short_softmax[0][:,0] + short_softmax[0][:,1]))

In [ ]:
print(len(short_softmax[0][:,0]))

In [ ]:
print(len(set(short_softmax[0][:,0] + short_softmax[0][:,1])))

In [ ]:
print(len(set(short_softmax[1][:,0] + short_softmax[1][:,1])))

In [ ]:
print(np.max(filtered_fq_mom))

### Plot vs Zenith

In [ ]:
fig, axes = plt.subplots(1,1,figsize=(9,5), facecolor='w')

plot_fixed_operating_performance(scores                 = short_softmax[0][:,0] + short_softmax[0][:,1], 
                                 labels                 = short_labels[0],  
                                 fixed_binning_features = filtered_fq_mom,
                                 fixed_bin_label        = 'Reconstructed Momentum',
                                 plot_binning_features  = filtered_test_zenith,
                                 plot_bin_label         = 'Zenith Angle',
                                 fpr_fixed_point        = 0.005, 
                                 index_dict             = label_dict, 
                                 fixed_bin_size         = 50, 
                                 plot_bins              = 10, 
                                 ax                     = axes,
                                 marker                 = 'o--',
                                 color                  = 'r',
                                 title_note             = '',
                                 metric                 = 'efficiency',
                                yrange                  = [0.85, 1.01]
                              )

plot_fixed_operating_performance(scores                 = short_softmax[1][:,0] + short_softmax[1][:,1], 
                                 labels                 = short_labels[1],  
                                 fixed_binning_features = filtered_fq_mom,
                                 fixed_bin_label        = 'Reconstructed Momentum',
                                 plot_binning_features  = filtered_test_zenith,
                                 plot_bin_label         = 'Zenith Angle',
                                 fpr_fixed_point        = 0.005, 
                                 index_dict             = label_dict, 
                                 fixed_bin_size         = 50, 
                                 plot_bins              = 10, 
                                 ax                     = axes,
                                 marker                 = 'o--',
                                 color                  = 'g',
                                 title_note             = '',
                                 metric                 = 'efficiency',
                                 yrange                 = [0.85, 1.01]
                              )

axes.legend(['ResNet (P(e-) + P(\u03B3))', 'FiTQun'])

### Plot vs Azimuth

In [ ]:
fig, axes = plt.subplots(1,1,figsize=(9,5), facecolor='w')

plot_fixed_operating_performance(scores                 = short_softmax[0][:,0] + short_softmax[0][:,1], 
                                 labels                 = short_labels[0],  
                                 fixed_binning_features = filtered_fq_mom,
                                 fixed_bin_label        = 'Reconstructed Momentum',
                                 plot_binning_features  = filtered_test_azimuth,
                                 plot_bin_label         = 'Azimuth Angle',
                                 fpr_fixed_point        = 0.005, 
                                 index_dict             = label_dict, 
                                 fixed_bin_size         = 50, 
                                 plot_bins              = 10, 
                                 ax                     = axes,
                                 marker                 = 'o--',
                                 color                  = 'r',
                                 title_note             = '',
                                 metric                 = 'efficiency',
                                yrange                  = [0.85, 1.01]
                              )

plot_fixed_operating_performance(scores                 = short_softmax[1][:,0] + short_softmax[1][:,1], 
                                 labels                 = short_labels[1],  
                                 fixed_binning_features = filtered_fq_mom,
                                 fixed_bin_label        = 'Reconstructed Momentum',
                                 plot_binning_features  = filtered_test_azimuth,
                                 plot_bin_label         = 'Azimuth Angle',
                                 fpr_fixed_point        = 0.005, 
                                 index_dict             = label_dict, 
                                 fixed_bin_size         = 50, 
                                 plot_bins              = 10, 
                                 ax                     = axes,
                                 marker                 = 'o--',
                                 color                  = 'g',
                                 title_note             = '',
                                 metric                 = 'efficiency',
                                 yrange                 = [0.85, 1.01]
                              )
axes.legend(['ResNet (P(e-) + P(\u03B3))', 'FiTQun'])

In [24]:
print(np.min(filtered_test_azimuth))
print(np.max(filtered_test_azimuth))

-3.1415873
3.1415908
